In [ ]:
import pandas as pd
import numpy as np
import gspread
import re
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from datetime import date

In [ ]:
#Declaring Google API
scope = ['https://spreadsheets.google.com/feeds',
            'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name ('/SELECTPATH/KEYNAME.json', scope)
gc = gspread.authorize(credentials)

In [ ]:
#Declaring Time variables
today = date.today().strftime('%m-%d-%Y')
ext = '.xlsx'
comp = today + ext

In [ ]:
#Data process 1 (jumlah UPV CMS) change file name to 'report.xlsx'
dat1 = pd.read_excel('/SELECTPATH/FILENAME1.xlsx')
dat1_inhouse = dat1[dat1['Agent Type'] == 'inhouse'].sort_values("Project").drop(labels=['Leads'], axis=1)
dat1_jv = dat1[dat1['Agent Type'] == 'inhouse_jv'].sort_values("Project")

In [ ]:
#filtering agent 
agent= pd.read_excel('/SELECTPATH/FILENAME2.xlsx')
active= agent[agent['Status'] != 'Nonactive']
email = active.drop(columns= ['Phone', 'Type', 'Captain', 'Office', 'Project', 'Tanggal Pembuatan', 'URL Seller Page'])
datmail_inhouse= pd.merge(dat1_inhouse, email, left_on='Referral Link', right_on='Unique Code').drop(columns='Nama_y')
datmail_jv= pd.merge(dat1_jv, email, left_on='Referral Link', right_on='Unique Code').drop(columns='Nama_y')

In [ ]:
#SAP data processing
dat2 = pd.read_csv('/SELECTPATH/FILENAME3.csv') #ganti path
dat2.columns.str.match("Unnamed")
dat22 =dat2.loc[:,~dat2.columns.str.match("Unnamed")]
dat2_2nd = dat22.drop(columns=['Source - ReportingX', 'Custom Status', 'Phone', 'Year Month (Lead Submission Date)'])
dat2_2nd['Source1tf'] = dat2_2nd['Lead Source Name'].str.findall('ECAT.+') #Regex
dat2_2nd['source2tf'] = dat2_2nd['Source'].str.findall('.+ecat.+') #Regex

In [ ]:
#filtering CONDITIONALS boolean
dat2_2nd['source_t/f'] = dat2_2nd['Source1tf'].astype(str) + '-' + dat2_2nd['source2tf'].astype(str)
dat3 = dat2_2nd[dat2_2nd['source_t/f'] != '[]-[]']
dat3_2nd = dat3.drop(columns=['Source1tf', 'source2tf'])

In [ ]:
#counting sales leads
dat3_2nd['Leads'] = dat3_2nd['E-Mail-Marketing'].map(dat3_2nd['E-Mail-Marketing'].value_counts())
dat3_email = dat3_2nd [['E-Mail-Marketing', 'Employee Responsible - Marketing', 'Leads']].drop_duplicates(subset='E-Mail-Marketing', keep='first')

In [ ]:
#Creating Customer Journey
ney = dat3_2nd
journey = []

#making conditionals
ney2 = ney[['All Leads', 'Accepted Leads', 'Reachable Leads', 'Follow Up Leads', 'Hot Prospect Leads', 'Won Leads']]
ney['sum'] = ney2.count(axis=1)

for row in ney['sum']:
    if row == 1:
        journey.append('All Leads')
    elif row == 2:
        journey.append('Accepted Leads')
    elif row == 3:
        journey.append('Reachable Leads')
    elif row == 4:
        journey.append('Follow Up Leads')
    elif row == 5:
        journey.append('Hot Prospect Leads')
    elif row == 6:
        journey.append('WON Leads') 

    else:
        journey.append('NO JOURNEY')
        
ney['Journey'] = journey

In [ ]:
#adding BSD team and projects
leadsacc1 = ney.drop(columns= ['All Leads', 'Accepted Leads', 'Reachable Leads', 'Follow Up Leads', 'Hot Prospect Leads', 'Won Leads', 'Leads', 'sum'])

leadsacc1['email-lower'] = leadsacc1['E-Mail-Marketing'].str.lower()
agentcode_in = datmail_inhouse[['Unique Code', 'Email']]
agentcode_in['email-lower'] = agentcode_in['Email'].str.lower()
leadsacc2 = pd.merge(leadsacc1, agentcode_in, how= 'left')

#project V2 agent
leadsv1 = pd.read_csv('/SELECTPATH/FILENAME4.csv')
leadsv2 = leadsv1[['Unique Code', 'Project V2']]
leadsv2['Project'] = leadsv2['Project V2']
sapmerge = pd.merge(leadsacc2, leadsv2, how= 'left')

#BSDTeam
team1 = pd.read_csv('/SELECTPATH/FILENAME5.csv')
team2 = team1[['Unique Code', 'BSD Team2']]
sapmerge2 = pd.merge(sapmerge, team2, how= 'left').drop(columns=['source_t/f', 'Project V2', 'email-lower']).rename(columns={'BSD Team2':'BSD Team'})

In [ ]:
#Team processing
dat3_email['email-lower'] = dat3_email['E-Mail-Marketing'].str.lower()
datmail_inhouse['email-lower'] = datmail_inhouse['Email'].str.lower()

merge_in= pd.merge(dat3_email, datmail_inhouse, how= 'outer')
merge_indrop = merge_in.drop(columns= ['E-Mail-Marketing', 'email-lower', 'Referral Link', 'Employee Responsible - Marketing']).dropna(subset= ['Unique Code'])
merge_indrop['UPV'] = merge_indrop['UPV']
merge_indrop['Leads'] = merge_indrop['Leads'].fillna(0)
merge_indrop2= merge_indrop[['Unique Code', 'Nama_x', 'Project', 'Agent Type', 'Email', 'Status', 'UPV', 'Leads']]

#changing project to project V2
leadsnoproj = leadsv2.drop(columns=['Project'])
merge_in2 = pd.merge(merge_indrop2, leadsnoproj, how='left')
#adding team
merge_in3 = pd.merge(merge_in2, team2, how= 'left').drop(columns= ['Project']).rename(columns= {'BSD Team2':'BSD Team', 'Project V2':'Project', 'Nama_x':'Nama Agent'})

In [ ]:
#JV CMS data processing
enq_jv = pd.read_excel('/SELECTPATH/FILENAME6.xlsx')
datmail_jv2 = datmail_jv[['Unique Code', 'Nama_x', 'Project', 'Agent Type', 'Email', 'Status', 'UPV', 'Leads']].rename(columns={'Nama_x':'Nama Agent'})
#datmail_jv2.loc['Total'] = datmail_jv2.select_dtypes(pd.np.number).sum()

#finding leads on enquiries
enq2 = enq_jv[enq_jv['Is Duplicate'] != 'YES']
enq22 = enq2[enq2['Agent Type'] == 'inhouse_jv']

#merging leads to jv only
enq2['Lead'] = enq2['Agent Email'].map(enq2['Agent Email'].value_counts())
enq3 = enq2[['Unique Code','Lead']].drop_duplicates(subset='Unique Code', keep= 'first')
jv = pd.merge(datmail_jv2, enq3, how= 'left').fillna(0).drop(columns=['Leads'])

In [ ]:
#Creating Score UPV Inhouse 
upv = merge_in3

UPVscore= []

for row in upv['UPV']:
    if row >= 100:
        UPVscore.append(4*4)
    elif row <= 99 and row >=30:
        UPVscore.append(4*3)
    elif row <= 29 and row >= 11:
        UPVscore.append(4*2)
    else:
        UPVscore.append(4)

upv['UPV score'] = UPVscore
#upv.head(5)

In [ ]:
#Creating score Leads Inhouse
upvleads = upv
leadscore = []

for row in upvleads['Leads']:
    if row >= 5:
        leadscore.append(4*4)
    elif row <= 4 and row >=3:
        leadscore.append(4*3)
    elif row <= 2 and row >= 1:
        leadscore.append(4*2)
    else:
        leadscore.append(4)

upvleads['Leads score'] = leadscore
#upvleads.head(5)

In [ ]:
#Changing journey won to number
won = sapmerge2[sapmerge2['Journey'] == 'WON Leads']
won['WON'] = won['Unique Code'].map(won['Unique Code'].value_counts()).drop_duplicates(keep= 'first')
won2 = won[['Unique Code', 'WON']]
upvwon= pd.merge(upvleads, won2, how= 'left').fillna(0)
upvwon= upvwon[['Unique Code', 'Nama Agent', 'Agent Type', 'Email', 'Project', 'BSD Team', 'Status', 'UPV', 'Leads', 'WON', 'UPV score', 'Leads score']]
#upvwon.head(5)

In [ ]:
#Creating score WON Inhouse 
upvleadswon = upvwon
wonscore = []

for row in upvleadswon['WON']:
    if row >= 1:
        wonscore.append(row*5)
    else:
        wonscore.append(0)

upvleadswon['WON score'] = wonscore
#upvleadswon.head(5)

In [ ]:
#Counting login sales
login= pd.read_excel('/Volumes/KINGSTON/Etang LLV/Report/December 2022/SalesGradingFile/agent-login-logs.xlsx')
login1= login[login['Success'] == 'Yes']
login1['Login per Week']= login1['Unique Code'].map(login1['Unique Code'].value_counts())
login2= login1[['Unique Code', 'Login per Week']].drop_duplicates(subset= 'Unique Code', keep= 'first')
logmerge= pd.merge(upvleadswon, login2, how= 'left').fillna(0)
#logmerge.head(5)

In [ ]:
#Creating score login Inhouse
upvleadswonlog = logmerge
logscore = []

for row in upvleadswonlog['Login per Week']:
    if row >= 5:
        logscore.append(1*4)
    elif row == 4:
        logscore.append(1*3)
    elif row == 3:
        logscore.append(1*2)
    else:
        logscore.append(1)

upvleadswonlog['Login score'] = logscore
#upvleadswonlog.head(5)

In [ ]:
#Creating Inhouse total and final score
totscore = upvleadswonlog
tot= []

total = totscore[['UPV score', 'Leads score', 'WON score', 'Login score']]
totscore['Total Score'] = total.sum(axis=1)

for row in totscore['Total Score']:
    if row >= 50:
        tot.append('A')
    elif row <= 49 and row >= 36:
        tot.append('B')
    elif row <= 35 and row >= 26:
        tot.append('C')
    elif row <= 25:
        tot.append('D')

totscore['Final Grade'] = tot

In [ ]:
#inhouse final file 
inhousejadi = totscore[['Unique Code', 'Nama Agent', 'Agent Type', 'Email', 'Project', 'BSD Team', 'Status', 'UPV', 'Leads','WON', 'Login per Week', 'UPV score', 'Leads score', 'WON score', 'Login score', 'Total Score',	'Final Grade']]

In [ ]:
#Creating Score UPV JV
jvUPV= []

for row in jv['UPV']:
    if row >= 100:
        jvUPV.append(4*4)
    elif row <= 99 and row >=30:
        jvUPV.append(4*3)
    elif row <= 29 and row >= 11:
        jvUPV.append(4*2)
    else:
        jvUPV.append(4)

jv['UPV score'] = jvUPV
#jv.head(5)


In [ ]:
#Creating score Leads JV
jvleads = []

for row in jv['Lead']:
    if row >= 5:
        jvleads.append(4*4)
    elif row <= 4 and row >=3:
        jvleads.append(4*3)
    elif row <= 2 and row >= 1:
        jvleads.append(4*2)
    else:
        jvleads.append(4)

jv['Leads score'] = jvleads
#jv.head(5)

In [ ]:
#Counting WON jv
jvwon = enq_jv[enq_jv['Status'] == 'Won']
jvwon['WON'] = jvwon['Unique Code'].map(jvwon['Unique Code'].value_counts()).drop_duplicates(keep= 'first')
jvwon2 = jvwon[['Unique Code', 'WON']]
jvwon3= pd.merge(jv, jvwon2, how= 'left').fillna(0) 
#jvwon3.head(5)

In [ ]:
#Creating Won score JV
wonscorejv= []

for row in jvwon3['WON']:
    if row >= 1:
        wonscorejv.append(row*5)
    else:
        wonscorejv.append(0)

jvwon3['WON score'] = wonscorejv
#jvwon3.head(5)

In [ ]:
#merging login JV
jvlogmerge= pd.merge(jvwon3, login2, how= 'left').fillna(0)
#jvlogmerge.head(5)

In [ ]:
#Creating Login score JV
jvlogscore = []

for row in jvlogmerge['Login per Week']:
    if row >= 5:
        jvlogscore.append(1*4)
    elif row == 4:
        jvlogscore.append(1*3)
    elif row == 3:
        jvlogscore.append(1*2)
    else:
        jvlogscore.append(1)

jvlogmerge['Login score'] = jvlogscore
#jvlogmerge.head(5)


In [ ]:
#Creating JV final Score 
sumjv= jvlogmerge
jvtot= []

jvtotal = sumjv[['UPV score', 'Leads score', 'WON score', 'Login score']]
sumjv['Total Score'] = jvtotal.sum(axis=1)

for row in sumjv['Total Score']:
    if row >= 50:
        jvtot.append('A')
    elif row <= 49 and row >= 36:
        jvtot.append('B')
    elif row <= 35 and row >= 26:
        jvtot.append('C')
    elif row <= 25:
        jvtot.append('D')

sumjv['Final Grade'] = jvtot
#sumjv.head(5)

In [ ]:
#jv final file
jvjadi = sumjv[['Unique Code', 'Nama Agent', 'Agent Type', 'Email', 'Project', 'Status', 'UPV', 'Lead','WON', 'Login per Week', 'UPV score', 'Leads score', 'WON score', 'Login score', 'Total Score',	'Final Grade']]

In [ ]:
#this is Worksheet ID
Sheet_id= '1VJJJMMOP3-CzS0GSzKRTFkZrVsrSFVFAaLy23UJ9Nb0'
#this is Sheet name 
Sheet_inhouse= 'Inhouse Performance '
Sheet_JV= 'Inhouse-JV Performance '

d2g.upload(inhousejadi, Sheet_id, Sheet_inhouse + today, credentials= credentials, row_names= False)
d2g.upload(jvjadi, Sheet_id, Sheet_JV + today, credentials= credentials, row_names= False)